In [1]:
!pip install wget

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190427140931-0000
KERNEL_ID = 1a7d1c78-7fe3-48df-a370-fdd6552550ed
  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/spark/shared/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
pyspark 2.3.0 requires py4j==0.10.6, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [2]:
!pip install tensorflow

    100% |################################| 92.5MB 217kB/s eta 0:00:01
    100% |################################| 368kB 4.3MB/s eta 0:00:01
    100% |################################| 3.2MB 2.8MB/s eta 0:00:01
    100% |################################| 102kB 4.2MB/s ta 0:00:01
    100% |################################| 17.2MB 968kB/s eta 0:00:01
    100% |################################| 61kB 2.6MB/s eta 0:00:01
    100% |################################| 1.2MB 3.9MB/s eta 0:00:01
    100% |################################| 61kB 2.5MB/s eta 0:00:01
    100% |################################| 2.1MB 3.1MB/s eta 0:00:01
    100% |################################| 61kB 2.8MB/s eta 0:00:01
    100% |################################| 337kB 4.5MB/s eta 0:00:01
    100% |################################| 92kB 4.0MB/s eta 0:00:01
    100% |################################| 583kB 4.3MB/s eta 0:00:01
    100% |################################| 2.8MB 3.1MB/s eta 0:00:01
    100% |#############

In [3]:
!pip install py4j==0.10.6

    100% |################################| 194kB 4.0MB/s eta 0:00:01


In [4]:
# The code was removed by Watson Studio for sharing.

[Row(ID=10642781, Case Number='HZ394208', Date='08/15/2016 11:15:00 AM', Block='061XX W GRAND AVE', IUCR='0460', Primary Type='BATTERY', Description='SIMPLE', Location Description='APARTMENT', Arrest=False, Domestic=False, Beat=2512, District=25, Ward=29, Community Area=19, FBI Code='08B', X Coordinate=1134907, Y Coordinate=1914405, Year=2016, Updated On='02/10/2018 03:50:01 PM', Latitude=41.921319589, Longitude=-87.779748093, Location='(41.921319589, -87.779748093)')]

In [5]:
df_data_1.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [6]:
df_data_1.dropna(how="any").count()

6140717

In [7]:
df_data_1.select('Primary Type').distinct().count()

35

In [8]:
df_data_1.select('Description').distinct().count()

381

In [9]:
df_data_1.select('Location Description').distinct().count()

180

In [10]:
df_data_1.select('Year').distinct().count()

19

In [11]:
df_data_1.select('IUCR').distinct().count()

403

In [12]:
df_data_1.select('Beat').distinct().count()

304

In [13]:
df_data_1.select('District').distinct().count()

25

In [14]:
df_data_1.select('Ward').distinct().count()

51

In [15]:
df_data_1.select('Community Area').distinct().count()

79

In [16]:
df_data_1.select('FBI Code').distinct().count()

26

In [17]:
splitted_data = df_data_1.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.dropna(how="any").count()))
print("Number of testing records : " + str(test_data.dropna(how="any").count()))
print("Number of prediction records : " + str(predict_data.dropna(how="any").count()))

Number of training records: 4911711
Number of testing records : 1106642
Number of prediction records : 122364


In [18]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.tuning import ParamGridBuilder;

In [19]:
stringIndexer_label = StringIndexer(inputCol="Primary Type", outputCol="label", handleInvalid="keep").fit(df_data_1.dropna(how="any"))
stringIndexer_fbi = StringIndexer(inputCol="FBI Code", outputCol="FBI_Code_IX", handleInvalid="keep")

In [20]:
vectorAssembler_features = VectorAssembler(inputCols=["FBI_Code_IX", "District", "Ward", "Community Area", "Year"], outputCol="features")

In [21]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In [22]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

In [23]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_fbi, vectorAssembler_features, rf, labelConverter])

In [24]:
train_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [25]:
model_rf = pipeline_rf.fit(train_data.dropna(how="any"))

In [26]:
predictions = model_rf.transform(test_data.dropna(how="any"))
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.789956
Test Error = 0.210044


In [27]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

In [28]:
service_path = 'https://us-south.ml.cloud.ibm.com'
instance_id = '0c7707e4-c298-40c9-a668-ae63f2cc31de'
username = 'd4ee1165-65d7-4b65-a3e2-ab66beea2657'
password = 'c2fe77bb-1508-494b-86b7-c50eb323f47b'

In [29]:
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

In [30]:
model_artifact = MLRepositoryArtifact(model_rf, training_data = train_data, name="Crime Line prediction")

In [31]:
saved_model = ml_repository_client.models.save(model_artifact)

In [32]:
saved_model.meta.available_props()

dict_keys(['label', 'evaluationMetrics', 'creationTime', 'pipelineVersionHref', 'modelType', 'authorEmail', 'inputDataSchema', 'trainingDataSchema', 'authorName', 'modelVersionHref', 'runtime', 'lastUpdated', 'evaluationMethod', 'version', 'trainingDataRef'])

In [33]:
print ("modelType: " + saved_model.meta.prop("modelType"))
print ("trainingDataSchema: " + str(saved_model.meta.prop("trainingDataSchema")))
print ("creationTime: " + str(saved_model.meta.prop("creationTime")))
print ("modelVersionHref: " + saved_model.meta.prop("modelVersionHref"))
print ("label: " + saved_model.meta.prop("label"))

modelType: sparkml-model-2.3
trainingDataSchema: {'fields': [{'nullable': True, 'type': 'integer', 'name': 'ID', 'metadata': {}}, {'nullable': True, 'type': 'string', 'name': 'Case Number', 'metadata': {}}, {'nullable': True, 'type': 'string', 'name': 'Date', 'metadata': {}}, {'nullable': True, 'type': 'string', 'name': 'Block', 'metadata': {}}, {'nullable': True, 'type': 'string', 'name': 'IUCR', 'metadata': {}}, {'nullable': True, 'type': 'string', 'name': 'Primary Type', 'metadata': {}}, {'nullable': True, 'type': 'string', 'name': 'Description', 'metadata': {}}, {'nullable': True, 'type': 'string', 'name': 'Location Description', 'metadata': {}}, {'nullable': True, 'type': 'boolean', 'name': 'Arrest', 'metadata': {}}, {'nullable': True, 'type': 'boolean', 'name': 'Domestic', 'metadata': {}}, {'nullable': True, 'type': 'integer', 'name': 'Beat', 'metadata': {}}, {'nullable': True, 'type': 'integer', 'name': 'District', 'metadata': {}}, {'nullable': True, 'type': 'integer', 'name': '

In [34]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

In [35]:
print (loadedModelArtifact.name)

Crime Line prediction


In [36]:
predictions = loadedModelArtifact.model_instance().transform(predict_data.dropna(how="any"))

In [37]:
predictions.show(5)

+-----+-----------+--------------------+--------------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----+-----------+--------------------+--------------------+--------------------+----------+--------------+
|   ID|Case Number|                Date|               Block|IUCR|Primary Type|        Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|label|FBI_Code_IX|            features|       rawPrediction|         probability|prediction|predictedLabel|
+-----+-----------+--------------------+--------------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------

In [38]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

+-------------------+-----+
|     predictedLabel|count|
+-------------------+-----+
|            ASSAULT| 8187|
|MOTOR VEHICLE THEFT| 1209|
|              THEFT|25943|
|            BATTERY|35886|
| DECEPTIVE PRACTICE| 2554|
|    CRIMINAL DAMAGE|14036|
|          NARCOTICS|11966|
|      OTHER OFFENSE|15174|
|           BURGLARY| 7409|
+-------------------+-----+



In [39]:
!pip install --upgrade plotly 

    100% |################################| 41.2MB 340kB/s eta 0:00:01
    100% |################################| 512kB 4.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
    100% |################################| 61kB 2.9MB/s eta 0:00:01
    100% |################################| 163kB 4.4MB/s eta 0:00:01
    100% |################################| 143kB 4.6MB/s eta 0:00:01
    100% |################################| 163kB 4.5MB/s eta 0:00:01
    100% |################################| 133kB 4.3MB/s eta 0:00:01
    100% |################################| 61kB 2.6MB/s eta 0:00:01
    100% |################################| 61kB 2.5MB/s eta 0:00:01
    100% |################################| 81kB 3.4MB/s eta 0:00:01
    100% |################################| 133kB 4.3MB/s eta 0:00:01
    100% |################################| 112kB 4.5MB/s eta 0:00:01
  Using c

In [40]:
!pip install --upgrade cufflinks==0.8.2

    100% |################################| 10.0MB 1.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/34/18/ef96260cb7a68cf613348cc7fe90efa4e32eaf23c46ade7032e6548e25a7/plotly-3.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3d/73/fe30c2daaaa0713420d0382b16fbb761409f532c56bdcc514bf7b6262bb6/pytz-2019.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f6/f3/cc6c6745347c1e997cc3e58390584a250b8e22b6dfc45414a7d69a3df016/numpy-1.16.3-cp35-cp35m-manylinux1_x86_64.whl
    100% |################################| 235kB 5.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/da/27/9a654d2b6cc1eaa517d1c5a4405166c7f6d72f04f6e7eea41855fe808a46/nbformat-4.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7d/e3/20f3d36

In [41]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go
init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]]))

In [42]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "District", "Ward", "Community Area", "FBI Code", "Year").toPandas()

In [43]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()
district_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['District'])]
district_layout = go.Layout(title='Predicted crime rate in the districts')

fig = go.Figure(data=district_data, layout=district_layout)
iplot(fig)

In [44]:
cumulative_stats = predictions_pdf.groupby(['District']).count()
district_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['predictedLabel'])]
district_layout = go.Layout(title='Predicted crime rate in the districts')

fig = go.Figure(data=district_data, layout=district_layout)
iplot(fig)

In [45]:
cumulative_stats = predictions_pdf.groupby(['Ward']).count()
ward_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['predictedLabel'])]
ward_layout = go.Layout(title='Predicted crime rate in the ward')

fig = go.Figure(data=ward_data, layout=ward_layout)
iplot(fig)

In [46]:
cumulative_stats = predictions_pdf.groupby(['Community Area']).count()
comm_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['predictedLabel'])]
comm_layout = go.Layout(title='Predicted crime rate in Community Area')

fig = go.Figure(data=comm_data, layout=comm_layout)
iplot(fig)

In [47]:
cumulative_stats1 = predictions_pdf.groupby(['FBI Code']).count()
fbi_data = [go.Pie(labels=cumulative_stats1.index, values=cumulative_stats1['predictedLabel'])]
fbi_layout = go.Layout(title='Predicted crime rate with the FBI Code')

fig = go.Figure(data=fbi_data, layout=fbi_layout)
iplot(fig)

In [48]:
year_data = [go.Bar(y=predictions_pdf.groupby(['predictedLabel']).mean()["Year"], x=cumulative_stats.index)]

year_layout = go.Layout(
    title='Number of Primary Type crimes taken place in the years',
    xaxis=dict(title = "Primary Type", showline=False),
    yaxis=dict(title = "Year"))

fig = go.Figure(data=year_data, layout=year_layout)
iplot(fig)

In [49]:
cumulative_stats = predictions_pdf.groupby(['Year']).count()
year_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['predictedLabel'])]
year_layout = go.Layout(title='Predicted crime rate based on the year')

fig = go.Figure(data=year_data, layout=year_layout)
iplot(fig)